# Link to the dataset 
https://www.kaggle.com/datasets/lama122/saudi-arabia-real-estate-aqar

# The import 

In [1]:
import pandas as pd

# Lecture

In [2]:
df = pd.read_csv('SA_Aqar.csv')

# Consultation

In [3]:
df.head(2)

,city,district,front,size,property_age,bedrooms,bathrooms,livingrooms,kitchen,garage,...,roof,pool,frontyard,basement,duplex,stairs,elevator,fireplace,price,details
0,الرياض,حي العارض,شمال,250,0,5,5,1,1,1,...,0,0,1,0,1,1,0,0,80000,للايجار فيلا دبلكس في موقع ممتاز جدا بالقرب من...
1,الرياض,حي القادسية,جنوب,370,0,4,5,2,1,1,...,0,0,1,0,0,1,0,0,60000,*** فيلا درج مع الصالة جديدة ***\n\nعبارة عن م...


In [4]:
df.isnull().sum()

city             0
district         0
front            0
size             0
property_age     0
bedrooms         0
bathrooms        0
livingrooms      0
kitchen          0
garage           0
driver_room      0
maid_room        0
furnished        0
ac               0
roof             0
pool             0
frontyard        0
basement         0
duplex           0
stairs           0
elevator         0
fireplace        0
price            0
details         80
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   city          3718 non-null   object
 1   district      3718 non-null   object
 2   front         3718 non-null   object
 3   size          3718 non-null   int64 
 4   property_age  3718 non-null   int64 
 5   bedrooms      3718 non-null   int64 
 6   bathrooms     3718 non-null   int64 
 7   livingrooms   3718 non-null   int64 
 8   kitchen       3718 non-null   int64 
 9   garage        3718 non-null   int64 
 10  driver_room   3718 non-null   int64 
 11  maid_room     3718 non-null   int64 
 12  furnished     3718 non-null   int64 
 13  ac            3718 non-null   int64 
 14  roof          3718 non-null   int64 
 15  pool          3718 non-null   int64 
 16  frontyard     3718 non-null   int64 
 17  basement      3718 non-null   int64 
 18  duplex        3718 non-null   int64 
 19  stairs

# Cleaning 

In [6]:
df.dropna(inplace=True)

# Test 
To test a json file retrieved from the web app 

In [7]:
filters = {'city': [' الخبر'], 'district': [], 'front': [], 'price': {'min': None, 'max': None}, 'size': {'min': None, 'max': None}, 'bedrooms': None, 'bathrooms': None, 'furnished': None, 'ac': None, 'roof': None, 'pool': None, 'frontyard': None, 'basement': None, 'duplex': None, 'stairs': None, 'elevator': None, 'fireplace': None}

In [8]:
filtered_df = df.copy()

### Apply location filters

In [9]:
if filters.get('city'):
    filtered_df = filtered_df[filtered_df['city'].isin(filters['city'])]
if filters.get('district'):
    filtered_df = filtered_df[filtered_df['district'].isin(filters['district'])]
if filters.get('front'):
    filtered_df = filtered_df[filtered_df['front'].isin(filters['front'])]


In [10]:
filtered_df.head(1)

,city,district,front,size,property_age,bedrooms,bathrooms,livingrooms,kitchen,garage,...,roof,pool,frontyard,basement,duplex,stairs,elevator,fireplace,price,details
2742,الخبر,حي العليا,شمال,325,14,4,5,2,1,0,...,0,0,1,0,1,1,0,0,65000,للإيجار دبلوكس متصل مرتب جداً باركيه ومكيفات ر...


### Apply price range filter

In [11]:
if filters.get('price'):
    price_min = filters['price'].get('min')
    price_max = filters['price'].get('max')
    if price_min is not None and price_min != '':
        filtered_df = filtered_df[filtered_df['price'] >= float(price_min)]
    if price_max is not None and price_max != '':
        filtered_df = filtered_df[filtered_df['price'] <= float(price_max)]

In [12]:
filtered_df.head(1)

,city,district,front,size,property_age,bedrooms,bathrooms,livingrooms,kitchen,garage,...,roof,pool,frontyard,basement,duplex,stairs,elevator,fireplace,price,details
2742,الخبر,حي العليا,شمال,325,14,4,5,2,1,0,...,0,0,1,0,1,1,0,0,65000,للإيجار دبلوكس متصل مرتب جداً باركيه ومكيفات ر...


### Filtering 'size', 'bedrooms', 'bathrooms' and 'livingrooms'

In [13]:
for field in ['size', 'bedrooms', 'bathrooms', 'livingrooms']:
    if filters.get(field) is not None:
        if isinstance(filters[field], dict):
            if 'min' in filters[field] and filters[field]['min'] is not None and filters[field]['min'] != '':
                try:
                    min_val = int(filters[field]['min'])
                    filtered_df = filtered_df[filtered_df[field] >= min_val]
                except ValueError:
                    pass
            if 'max' in filters[field] and filters[field]['max'] is not None and filters[field]['max'] != '':
                try:
                    max_val = int(filters[field]['max'])
                    filtered_df = filtered_df[filtered_df[field] <= max_val]
                except ValueError:
                    pass
        else:
            # Handle single value
            if filters[field] is not None and filters[field] != '':
                try:
                    value = int(filters[field])
                    filtered_df = filtered_df[filtered_df[field] >= value]
                except ValueError:
                    pass

### Apply boolean filters and sort the data

In [14]:
for col in ['furnished', 'ac', 'roof', 'pool', 'frontyard', 'basement', 
            'duplex', 'stairs', 'elevator', 'fireplace']:
    if filters.get(col) is not None:
        filtered_df = filtered_df[filtered_df[col] == True] if filters[col] else filtered_df[filtered_df[col] == False]

filtered_df = filtered_df.sort_values('price')

### It works

In [15]:
filtered_df.head(1)

,city,district,front,size,property_age,bedrooms,bathrooms,livingrooms,kitchen,garage,...,roof,pool,frontyard,basement,duplex,stairs,elevator,fireplace,price,details
2770,الخبر,حي السفن,شمال,525,21,4,2,1,1,0,...,0,0,1,0,0,0,0,0,30000,لكم فله راح عندي لكم فله تمام جديده اربع غرف و...
